In [ ]:
!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch torchaudio torchvision torchtext torchdata

Found existing installation: torch 2.1.0+cu121
Uninstalling torch-2.1.0+cu121:
  Successfully uninstalled torch-2.1.0+cu121
Found existing installation: torchaudio 2.1.0+cu121
Uninstalling torchaudio-2.1.0+cu121:
  Successfully uninstalled torchaudio-2.1.0+cu121
Found existing installation: torchvision 0.16.0+cu121
Uninstalling torchvision-0.16.0+cu121:
  Successfully uninstalled torchvision-0.16.0+cu121
Found existing installation: torchtext 0.16.0
Uninstalling torchtext-0.16.0:
  Successfully uninstalled torchtext-0.16.0
Found existing installation: torchdata 0.7.0
Uninstalling torchdata-0.7.0:
  Successfully uninstalled torchdata-0.7.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [26]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [27]:
import os

In [28]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/gdrive/MyDrive/Document Compression Dataset/Raw Corpus/train_set.csv')

df = df.drop_duplicates()
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()

df = df.iloc[:10000, :]

In [29]:
df

,source sentence,target sentence
0,"Fr xmpl, cntrs wth lt f dsrts cn trnsfrm thr ...","For example, countries with a lot of deserts c..."
1,"s th nmbr f ppl grws, th nd fr hbtbl nvrnmnt ...","As the number of people grows, the need for a ..."
2,Bsds sm tchnlgcl dtrmnsts tht llw th dvlpmnt f...,Besides some technological determinists that a...
3,Sfty s n f th crcl prblms tht mny cntrs nd cmp...,Safety is one of the crucial problems that man...
4,"n th n hnd, mr nd mr vrss nd hckrs cn ccss prs...","On the one hand, more and more viruses and hac..."
...,...,...
10013,"Ths tpc lds m t thnk f th prfssnl skll ""rchtct...",This topic leads me to think of the profession...
10014,"n th thr hnd, th skyscrpr cs mr prblms, sch s ...","On the other hand, the skyscraper cause more p..."
10015,"s sn s rd th lttr bt yr prblm, dcdd t wrt y ...",As soon as I read the letter about your proble...
10016,"Fr nstnc, sgnng p fr hp-hp dnc r rbc clsss, s ...","For instance, signing up for hip-hop dance or ..."


In [ ]:
!pip install revtok

In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

In [ ]:
import torchtext
import revtok
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer('subword', language='en')
tokens = tokenizer("and when you improve searchability , you actually take away the one advantage of print , which is serendipity .")

In [ ]:
tokens

[' and ',
 ' when ',
 ' you ',
 ' improve ',
 ' searchability ',
 ' , ',
 ' you ',
 ' actually ',
 ' take ',
 ' away ',
 ' the ',
 ' one ',
 ' advantage ',
 ' of ',
 ' print ',
 ' , ',
 ' which ',
 ' is ',
 ' serendipity ',
 ' . ']

In [ ]:
from torchtext.transforms import BERTTokenizer
VOCAB_FILE = "https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt"
tokenizer = BERTTokenizer(vocab_path=VOCAB_FILE, do_lower_case=True, return_tokens=True)

100%|██████████| 232k/232k [00:00<00:00, 2.03MB/s]


In [ ]:
tokenizer("and when you improve searchability , you actually take away the one advantage of print , which is serendipity .")

['and',
 'when',
 'you',
 'improve',
 'search',
 '##ability',
 ',',
 'you',
 'actually',
 'take',
 'away',
 'the',
 'one',
 'advantage',
 'of',
 'print',
 ',',
 'which',
 'is',
 'ser',
 '##end',
 '##ip',
 '##ity',
 '.']


# Language Translation with ``nn.Transformer`` and torchtext

This tutorial shows:
    - How to train a translation model from scratch using Transformer.
    - Use torchtext library to access  [Multi30k](http://www.statmt.org/wmt16/multimodal-task.html#task1)_ dataset to train a German to English translation model.


## Data Sourcing and Processing

[torchtext library](https://pytorch.org/text/stable/)_ has utilities for creating datasets that can be easily
iterated through for the purposes of creating a language translation
model. In this example, we show how to use torchtext's inbuilt datasets,
tokenize a raw text sentence, build vocabulary, and numericalize tokens into tensor. We will use
[Multi30k dataset from torchtext library](https://pytorch.org/text/stable/datasets.html#multi30k)_
that yields a pair of source-target raw sentences.

To access torchtext datasets, please install torchdata following instructions at https://github.com/pytorch/data.




In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List


# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# Place-holders
token_transform = {}
vocab_transform = {}

Create source and target language tokenizer. Make sure to install the dependencies.

```python
pip install -U torchdata
pip install -U spacy
python -m spacy download en_core_web_sm
python -m spacy download de_core_news_sm
```


In [ ]:
# !pip install -U torchdata
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

2024-01-23 05:31:08.389051: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 05:31:08.389123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 05:31:08.390794: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 05:31:08.399335: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-23 05:31:09.675601: W tensorflow/compiler/tf2

In [ ]:
!pip install portalocker

In [ ]:
!pip install revtok

In [ ]:
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    print(f"ln: {ln}")
    # Training data Iterator
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

ln: de
ln: en


In [ ]:
train_iter1 = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

In [30]:
total_rows = len(df)
split_ratio = 0.8

train_df = df.iloc[:int(total_rows * split_ratio), :].reset_index(drop=True)
test_df = df.iloc[int(total_rows * split_ratio):, :].reset_index(drop=True)

In [31]:
train_source_sentences = list(train_df['source sentence'].values)
train_target_sentences = list(train_df['target sentence'].values)

test_source_sentences = list(test_df['source sentence'].values)
test_target_sentences = list(test_df['target sentence'].values)

In [32]:
from torchtext.transforms import BERTTokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
from typing import Iterable, List

VOCAB_FILE = "https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt"
bert_tokenizer = BERTTokenizer(vocab_path=VOCAB_FILE, do_lower_case=True, return_tokens=True)

# erroneous_sources = list(df['source sentence'].values)
# correct_targets = list(df['target sentence'].values)

SRC_LANGUAGE = 'src_en'
TGT_LANGUAGE = 'tgt_en'

token_transform = {}
vocab_transform = {}

# BERT tokenizer for both source and target languages
token_transform[SRC_LANGUAGE] = bert_tokenizer
token_transform[TGT_LANGUAGE] = bert_tokenizer

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

# Create torchtext's Vocab object for erroneous sources using BERT tokenizer
vocab_transform[SRC_LANGUAGE] = build_vocab_from_iterator([bert_tokenizer(sent) for sent in train_source_sentences],
                                                         min_freq=1,
                                                         specials=special_symbols,
                                                         special_first=True)

# Create torchtext's Vocab object for correct targets using BERT tokenizer
vocab_transform[TGT_LANGUAGE] = build_vocab_from_iterator([bert_tokenizer(sent) for sent in train_target_sentences],
                                                         min_freq=1,
                                                         specials=special_symbols,
                                                         special_first=True)

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)

# Example usage:
# Access tokens for erroneous sources and correct targets
example_idx = 0
src_tokens = list(token_transform[SRC_LANGUAGE](train_source_sentences[example_idx]))
tgt_tokens = list(token_transform[TGT_LANGUAGE](train_target_sentences[example_idx]))

print(f"Source Tokens ({SRC_LANGUAGE}): {src_tokens}")
print(f"Target Tokens ({TGT_LANGUAGE}): {tgt_tokens}")

100%|██████████| 232k/232k [00:00<00:00, 10.7MB/s]


Source Tokens (src_en): ['fr', 'x', '##mp', '##l', ',', 'cn', '##tr', '##s', 'w', '##th', 'lt', 'f', 'ds', '##rts', 'cn', 'tr', '##ns', '##fr', '##m', 'th', '##r', 'ds', '##rt', 't', 'nc', '##rs', 'th', '##r', 'h', '##bt', '##bl', 'l', '##nd', 'n', '##d', 's', 'rr', '##gt', '##n', 't', 'pr', '##vd', 'cl', '##n', 'w', '##tr', 't', 'th', 'ds', '##rt', '.']
Target Tokens (tgt_en): ['for', 'example', ',', 'countries', 'with', 'a', 'lot', 'of', 'deserts', 'can', 'transform', 'their', 'desert', 'to', 'increase', 'their', 'habit', '##able', 'land', 'and', 'use', 'irrigation', 'to', 'provide', 'clean', 'water', 'to', 'the', 'desert', '.']


In [ ]:
from torchtext.transforms import BERTTokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import Multi30k
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# Define BERT tokenizer for the target language
VOCAB_FILE = "https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt"
bert_tokenizer = BERTTokenizer(vocab_path=VOCAB_FILE, do_lower_case=True, return_tokens=True)

# Create a data iterator for the Multi30k dataset
train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

# Define a function to yield tokens from the iterator using BERT tokenizer
def yield_bert_tokens(data_iter, language):
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}
    for data_sample in data_iter:
        yield bert_tokenizer(data_sample[language_index[language]])

# Build vocabulary using BERT tokenizer
vocab_transform_tgt = build_vocab_from_iterator(yield_bert_tokens(train_iter, TGT_LANGUAGE),
                                               min_freq=1,
                                               specials=special_symbols,
                                               special_first=True)

# Set ``UNK_IDX`` as the default index for target vocabulary
vocab_transform_tgt.set_default_index(UNK_IDX)

# Iterate over the training data and print BERT tokens for the target sentences
for example in train_iter:
    src_tokens = list(yield_tokens([example], SRC_LANGUAGE))[0]
    tgt_bert_tokens = list(yield_bert_tokens([example], TGT_LANGUAGE))[0]

    print(f"Source Tokens ({SRC_LANGUAGE}): {src_tokens}")
    print(f"Target BERT Tokens ({TGT_LANGUAGE}): {tgt_bert_tokens}")
    print()

100%|██████████| 232k/232k [00:00<00:00, 2.00MB/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/iter/combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


Streaming output truncated to the last 5000 lines.
Target BERT Tokens (en): ['five', 'people', 'wearing', 'black', 'and', 'holding', 'musical', 'instruments', 'looking', 'upwards', '.']

Source Tokens (de): ['Eine', 'Frau', 'in', 'einem', 'karierten', 'Kleid', 'führt', 'barfuß', 'einen', 'Ritualtanz', 'über', 'Bambusstangen', 'auf', '.']
Target BERT Tokens (en): ['a', 'woman', 'in', 'a', 'plaid', 'dress', 'and', 'bare', 'feet', 'performing', 'a', 'ritual', 'dance', 'over', 'bamboo', 'poles', '.']

Source Tokens (de): ['Die', 'Ansicht', 'eines', 'Hinterkopfs', 'einer', 'Person', ',', 'die', 'sich', 'den', 'Kopf', 'kratzt', '.']
Target BERT Tokens (en): ['the', 'back', 'of', 'a', 'person', "'", 's', 'head', 'is', 'shown', ',', 'as', 'they', 'scratch', 'their', 'head', 'with', 'their', 'hand', '.']

Source Tokens (de): ['Ein', 'Mann', 'in', 'orange-schwarzer', 'Kleidung', 'läuft', 'bei', 'einem', 'Straßenmarathon', 'an', 'Zuschauern', 'vorbei', '.']
Target BERT Tokens (en): ['a', 'man', '

## Seq2Seq Network using Transformer

Transformer is a Seq2Seq model introduced in [“Attention is all you
need”](https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)_
paper for solving machine translation tasks.
Below, we will create a Seq2Seq network that uses Transformer. The network
consists of three parts. First part is the embedding layer. This layer converts tensor of input indices
into corresponding tensor of input embeddings. These embedding are further augmented with positional
encodings to provide position information of input tokens to the model. The second part is the
actual [Transformer](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html)_ model.
Finally, the output of the Transformer model is passed through linear layer
that gives unnormalized probabilities for each token in the target language.




In [33]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

During training, we need a subsequent word mask that will prevent the model from looking into
the future words when making predictions. We will also need masks to hide
source and target padding tokens. Below, let's define a function that will take care of both.




In [34]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

Let's now define the parameters of our model and instantiate the same. Below, we also
define our loss function which is the cross-entropy loss and the optimizer used for training.




In [35]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=5e-5, betas=(0.9, 0.98), eps=1e-9)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


## Collation

As seen in the ``Data Sourcing and Processing`` section, our data iterator yields a pair of raw strings.
We need to convert these string pairs into the batched tensors that can be processed by our ``Seq2Seq`` network
defined previously. Below we define our collate function that converts a batch of raw strings into batch tensors that
can be fed directly into our model.




In [36]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

Let's define training and evaluation loop that will be called for each
epoch.




In [37]:
! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

In [38]:
import os
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
import math
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.transforms import BERTTokenizer
from transformers import AdamW
import sys

In [39]:
class TextDataset(Dataset):

    def __init__(self, source_sentences, target_sentences):
        self.source_sentences = source_sentences
        self.target_sentences = target_sentences

    def __len__(self):
        return len(self.source_sentences)

    def __getitem__(self, idx):
        return self.source_sentences[idx], self.target_sentences[idx]

In [40]:
train_dataset = TextDataset(train_source_sentences, train_target_sentences)
test_dataset = TextDataset(test_source_sentences, test_target_sentences)

In [41]:
train_dataset

In [42]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    # train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0

    # val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

Now we have all the ingredients to train our model. Let's do it!




In [43]:
from timeit import default_timer as timer
NUM_EPOCHS = 5

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch: 1, Train loss: 7.879, Val loss: 7.130, Epoch time = 30.146s
Epoch: 2, Train loss: 6.637, Val loss: 6.327, Epoch time = 28.055s
Epoch: 3, Train loss: 6.118, Val loss: 6.111, Epoch time = 30.366s
Epoch: 4, Train loss: 5.938, Val loss: 5.975, Epoch time = 30.270s
Epoch: 5, Train loss: 5.749, Val loss: 5.806, Epoch time = 28.421s


In [44]:
print(translate(transformer, "thgh sh cmpltd hr ltrtr rvw, sh stll ndd t wrk n hr mthds sctn."))

 the , and the and the and and the and and and and and and and and and and and and . 


A group of people standing in front of a crowd .

## References

1. Attention is all you need paper.
   https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf
2. The annotated transformer. https://nlp.seas.harvard.edu/2018/04/03/attention.html#positional-encoding

